# LECL Basic Code

- Using  Ollama model

ref : https://python.langchain.com/docs/expression_language/why/#lcel

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_community.chat_models import ChatOllama

In [5]:
# prompt 생성
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)

In [8]:
output_parser = StrOutputParser()
model = ChatOllama() # llama2

model

ChatOllama()

In [11]:
chain = (
    {"topic" : RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

chain.invoke("F1 drivers")

'\nWhy did the F1 driver break up with his girlfriend?\n\nShe was always turning left!'

## Runnable
- RunnablePassthrough
- RunnableParallel
- RunnableLambda

In [20]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

prompt = ChatPromptTemplate.from_template("{country} 의 수도는 어디야 ?")
model = ChatOllama() # llama2

In [22]:
# RunnablePassthrough
chain = (
    {"country" : RunnablePassthrough() }
    | prompt 
    | model
    | output_parser
)

chain.invoke("대한민국")

'\n대한민국의 수도는  Séoul (서울)이다.'

In [27]:
# RunnableParallel

prompt1 = ChatPromptTemplate.from_template("{country} 의 수도는 어디야 ?")
prompt2 = ChatPromptTemplate.from_template("{country} 의 인구는 몇명이야 ?")

chain1 = (
    {"country" : RunnablePassthrough() }
    | prompt1
    | model
    | output_parser
)

chain2 = (
    {"country" : RunnablePassthrough() }
    | prompt2
    | model
    | output_parser
)

map_chain = RunnableParallel(수도=chain1, 인구=chain2)


chain.invoke("대한민국")

{'수도': '\nThe capital of South Korea is Seoul.',
 '인구': '\nAs of 2023, the estimated population of South Korea is around 51.7 million people.'}

In [28]:
# RunnableLambda

def combine_text(text):
    return text["수도"] + " " + text["인구"]

lambda_chain = (
    map_chain
    | RunnableLambda(combine_text)
    | StrOutputParser()
)

lambda_chain.invoke("대한민국")

'\nThe capital of South Korea is Seoul. \nAs of December 2022, the estimated population of South Korea is around 51.7 million people.'

In [40]:
# RAG 의 경우
rag_chain = (
    map_chain
    | {"info" : RunnableLambda(combine_text)}
    | ChatPromptTemplate.from_template("다음의 내용을 한국어로 번역하고, 적절한 곳에 적절한 이모티콘을 추가해줘 : {info}")
    | model
)

rag_chain.invoke("대한민국").content

'\n서울이 대한민국의 수도다.\n국립통계위원회(Kostat)에 따라 最新의 인구조사를 통해, 2022년 대한민국의 인구가 약 5180만명으로 Estimated to be.\n\n🇰🇷🏙️'